In [1]:
from functools import cache
import numpy as np
import itertools
import heapq

In [2]:
test_input = '''1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
'''

In [3]:
puzzle_input = open('inputs/15').read().strip()

In [4]:
def string_to_2d(s):
    # add spaces between nums
    s = '\n'.join(map(lambda x: ' '.join(list(x)), s.split('\n'))).strip()
    rows = s.count('\n') + 1
    return np.fromstring(s, sep=' ').reshape(rows, -1).astype(np.int64)

In [5]:
def in_bounds(p, shape):
    return p[0] >= 0 and p[0] < shape[0] and p[1] >= 0 and p[1] < shape[1]

In [6]:
def all_neighbors(p): 
    a, b = p
    return [(a+1, b), (a, b+1), (a-1, b), (a, b-1)]

In [7]:
# this is retarded, but fix later
def magnify_board(board, tile_num=5):
    d = board.shape[0]
    
    tiled = np.tile(board, (tile_num, tile_num))
    
    for i in range(tile_num*d):
        for j in range(tile_num*d):        
            increase = i // d + j // d - 1

            tiled[i, j] = (tiled[i, j] + increase) % 9 + 1
    
    return tiled

In [8]:
# followed this for making a priority queue in python

# https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/
# involves tricks with priority queues: pushing same value multiple times
# and obviates need for the unvisited set

def solve_board(board):
    # square board
    d = board.shape[0]
        
    start = (0, 0)
    
    unvisited = set(itertools.product(range(d), range(d)))

    distances = np.full(board.shape, np.inf)
    distances[start] = 0

    pq = [(0, start)]
    
    while pq:
        current_distance, current_vertex = heapq.heappop(pq)
        
        # Nodes can get added to the priority queue multiple times. We only
        # process a vertex the first time we remove it from the priority queue.
        if current_vertex not in unvisited:
            continue

        neighbors = filter(lambda p: p in unvisited and in_bounds(p, board.shape), all_neighbors(current_vertex))

        for n in neighbors:
            new_tentative = board[n] + current_distance
            if new_tentative < distances[n]:
                distances[n] = new_tentative
                heapq.heappush(pq, (new_tentative, n))
        
        unvisited.remove(current_vertex)

    return distances[d-1, d-1].astype(int)

In [9]:
def p1(puzzle_input):
    board = string_to_2d(puzzle_input)

    return solve_board(board)

In [10]:
def p2(puzzle_input):
    board = string_to_2d(puzzle_input)
    board = magnify_board(board)

    return solve_board(board)

In [11]:
assert p1(test_input) == 40

In [12]:
assert p1(puzzle_input) == 696

In [13]:
assert p2(test_input) == 315

In [14]:
assert p2(puzzle_input) == 2952